In [1]:
import pandas as pd

In [50]:
df_capital = pd.read_csv('datasets/lacapital.csv')

df_capital.dropna(inplace=True)

# Limpieza y normalización de texto 

In [55]:
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('spanish'))

def remove_accents(input_str):
  nfkd_form = unicodedata.normalize('NFKD', input_str)
  return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

def remove_stopwords(text):
  word_tokens = word_tokenize(text)
  filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
  return " ".join(filtered_text)

for col in df_capital.columns[:-2]:
    df_capital[col] = df_capital[col].str.lower()
    df_capital[col] = df_capital[col].str.replace('[^\w\s]', '', regex=True)
    df_capital[col] = df_capital[col].apply(remove_stopwords)
    df_capital[col] = df_capital[col].apply(remove_accents)

[nltk_data] Downloading package stopwords to /home/isaias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/isaias/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [56]:
df_capital

,titulo,texto,categoria,url,fecha
0,tiempo rosario sabado calor anticipo lluvias,servicio meteorologico nacional smn indico pro...,ciudad,https://www.lacapital.com.ar/la-ciudad/el-tiem...,2023-10-20T22:53:00
1,provincial debuta sabado liga argentina,provincial debutara sabado 22 salvador bonilla...,ovacion,https://www.lacapital.com.ar/ovacion/provincia...,2023-10-20T22:20:00
2,efecto sequia economia santa fe cayo 76,actividad economica santa fe cayo 76 julio acu...,economia,https://www.lacapital.com.ar/economia/por-efec...,2023-10-20T22:11:00
3,tensa tregua cambiaria semana preelectoral,refuerzo swap chino fuerzas seguridad mercado ...,economia,https://www.lacapital.com.ar/economia/tensa-tr...,2023-10-20T22:09:00
4,plaza juega salvacion top 12 urba,sabado finaliza fase regular top 12 jornada de...,ovacion,https://www.lacapital.com.ar/ovacion/plaza-jue...,2023-10-20T22:05:00
...,...,...,...,...,...
210,quedo preso femicida mato pareja pension barri...,julio alberto ramirez changarin mercado produc...,policiales,https://www.lacapital.com.ar/policiales/quedo-...,2023-10-21T17:30:00
211,atilra salio cruce sancor videos muestran cami...,comunicado difundido sabado consejo directivo ...,informacion general,https://www.lacapital.com.ar/informacion-gener...,2023-10-21T17:23:00
212,confirman asesinato noveno argentino manos ham...,argentina sumo novena victima fatal menos grup...,mundo,https://www.lacapital.com.ar/el-mundo/confirma...,2023-10-21T17:16:00
213,juegos panamericanos pacheco remo metio final,jornada sabatina bien tempranito rosarino igna...,ovacion,https://www.lacapital.com.ar/ovacion/juegos-pa...,2023-10-21T16:33:00


# Ejercicio 2

In [57]:
df_capital['categoria'].value_counts()

categoria
ovacion                46
politica               34
ciudad                 25
zoom                   21
policiales             19
region                 16
informacion general    13
economia               11
mundial rugby 2023      8
mundo                   5
educacion               4
turismo                 4
negocios                2
colon                   1
julian montoya          1
baby etchecopar         1
opinion                 1
Name: count, dtype: int64

In [101]:
categorias = df_capital['categoria'].value_counts()
categorias = categorias[categorias >= 10].index[:4]

In [102]:
categorias_df = list()

for categoria in categorias:
    temp_df = df_capital[df_capital['categoria'] == categoria]
    categorias_df.append(temp_df.sample(10, random_state=1))

In [103]:
x = list()
y = list()

for df in categorias_df:
    x += df['titulo'].tolist()
    y += df['categoria'].tolist()

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

In [105]:
x_train, x_test, y_train, y_test = \
    train_test_split(x, y, test_size=0.2, random_state=1)

In [106]:
vectorizer = TfidfVectorizer()
x_train_vectorized = vectorizer.fit_transform(x_train)
x_test_vectorized = vectorizer.transform(x_test)

modelo_LR = LogisticRegression(max_iter=1000)
modelo_LR.fit(x_train_vectorized, y_train)

LogisticRegression(max_iter=1000)

In [107]:
y_pred_LR = modelo_LR.predict(x_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

In [108]:
print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística: 0.5
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

      ciudad       1.00      0.25      0.40         4
     ovacion       1.00      0.50      0.67         2
    politica       0.00      1.00      0.00         0
        zoom       1.00      1.00      1.00         2

    accuracy                           0.50         8
   macro avg       0.75      0.69      0.52         8
weighted avg       1.00      0.50      0.62         8



In [109]:
print(modelo_LR.predict(vectorizer.transform(["en pesos", "pesos de los pumas",
                                              "pesos de los pumas con milei",
                                              "pesos de los pumas con milei en un show de cine"])))

['politica' 'ovacion' 'politica' 'zoom']


# Ejercicio 3

# Ejercicio 4